In [ ]:
#Joshua Pan DIS Apr 2022
#Scrape IAState site to turn one (or multiple) month of hourly weather data into a csv
import datetime as dt
from dateutil.relativedelta import *
import urllib.request
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive/')
%ls
%cd /content/drive/MyDrive/DIS\ Big\ Data/Final\ Proj\ Code

Mounted at /content/drive/
drive/  sample_data/
/content/drive/MyDrive/DIS Big Data/Final Proj Code


In [ ]:
def getASOSmonth(ASOSstn, year, month):
    """
    From the Iowa State Mesonet website, pull a month of data logged by an ASOS station.
    
    :Parameters:
        ASOSstn (str): ICAO or four-char code of ASOS station (IATA/three-char in the US)
        
        year (int)
        
        month (int)
    
    :rtype: pandas dataframe with timestamps and values of wx vars
    """
    
    #generate the url to scrape
    startdate = dt.datetime(year, month, 1)
    enddate = startdate + relativedelta(months = 1)
    url = 'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={}&data=all&year1={}&month1={}&day1={}&year2={}&month2={}&day2={}&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=2'.format(ASOSstn, startdate.year, startdate.month, startdate.day, enddate.year, enddate.month, enddate.day)
    print(url)
    
    #parse lines of comma-separated vars into df
    asosfile = urllib.request.urlopen(url)
    lines = [str(line) for line in asosfile.readlines()]
    df = pd.DataFrame(columns=lines[0].split(','), 
                      data=[row.split(',') for row in lines[1:]])
    
    return df

In [ ]:
#save scraped weather data
stn = 'EKYT'
yrs = range(2022, 2023)
for yr in yrs:
    for mo in range(1, 5):
        df = getASOSmonth(stn, yr, mo)
        df.to_csv(path_or_buf = './rawwx/{}{:02d}_{}_rawwx.csv'.format(yr, mo, stn))

https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=EKYT&data=all&year1=2022&month1=1&day1=1&year2=2022&month2=2&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=2
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=EKYT&data=all&year1=2022&month1=2&day1=1&year2=2022&month2=3&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=2
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=EKYT&data=all&year1=2022&month1=3&day1=1&year2=2022&month2=4&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=2
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=EKYT&data=all&year1=2022&month1=4&day1=1&year2=2022&month2=5&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=2


In [ ]:
df.columns
df

,b'station,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth\n'
0,b'EKYT,2022-04-01 00:20,28.40,21.20,74.08,360.00,5.00,0.00,29.94,M,...,M,M,M,M,M,M,M,22.12,EKYT 010020Z AUTO 36005KT 9999 NCD M02/M06 Q1014,M\n'
1,b'EKYT,2022-04-01 00:50,28.40,21.20,74.08,10.00,5.00,0.00,29.97,M,...,M,M,M,M,M,M,M,22.12,EKYT 010050Z AUTO 01005KT 9999 NCD M02/M06 Q1015,M\n'
2,b'EKYT,2022-04-01 01:20,28.40,23.00,79.92,20.00,5.00,0.00,29.97,M,...,M,M,M,M,M,M,M,22.12,EKYT 010120Z AUTO 02005KT 9999 NCD M02/M05 Q1015,M\n'
3,b'EKYT,2022-04-01 01:50,28.40,23.00,79.92,10.00,5.00,0.00,29.97,M,...,M,M,M,M,M,M,M,22.12,EKYT 010150Z AUTO 01005KT 9999 NCD M02/M05 Q1015,M\n'
4,b'EKYT,2022-04-01 02:20,30.20,23.00,74.26,10.00,6.00,0.00,29.97,M,...,M,M,M,M,M,M,M,23.36,EKYT 010220Z AUTO 01006KT 9999 NCD M01/M05 Q1015,M\n'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,b'EKYT,2022-04-30 21:50,44.60,33.80,65.61,0.00,0.00,0.00,30.21,M,...,M,M,M,M,M,M,M,44.60,EKYT 302150Z AUTO 00000KT 9999 BKN060/// 07/01...,M\n'
2005,b'EKYT,2022-04-30 22:20,44.60,35.60,70.48,90.00,2.00,0.00,30.21,M,...,M,M,M,M,M,M,M,44.60,EKYT 302220Z AUTO 09002KT 9999 BKN060/// 07/02...,M\n'
2006,b'EKYT,2022-04-30 22:50,44.60,35.60,70.48,100.00,3.00,0.00,30.18,M,...,M,M,M,M,M,M,M,43.09,EKYT 302250Z AUTO 10003KT 9999 BKN060/// 07/02...,M\n'
2007,b'EKYT,2022-04-30 23:20,44.60,37.40,75.68,0.00,0.00,0.00,30.21,M,...,M,M,M,M,M,M,M,44.60,EKYT 302320Z AUTO 00000KT 9999 BKN060/// 07/03...,M\n'


In [ ]:
#functions to compute daily weather features
pcptypes = {'RA', 'SN', 'RASN', 'SNRA'}

def pcpint(wxcode):
    #Helper method to give precip intensity (0: none, 1: light, 2: moderate, 3: heavy) based on METAR wxcode (e.g., +RA)
    if wxcode in pcptypes:
        return 2
    if len(wxcode) > 1 and wxcode[1:] in pcptypes:
        return 1 if wxcode[0] == '-' else 3
    return 0

def dailywx(rawdf):
    """
    Given the dataframe produced by getASOSmonth(), produce a new dataframe containing daily summary wx vars (e.g., max temp).
    Compute theses summaries using helper methods.
    
    :Parameters:
        rawdf (pd df): pandas dataframe with timestamps and values of wx vars
    
    :rtype: pandas dataframe with daily summarized wx vars
    """
    
    rawdf['validdt'] = rawdf.apply(lambda row: dt.datetime.strptime(row['valid'], '%Y-%m-%d %H:%M'), axis = 1)
    
    startdate = rawdf['validdt'].iloc[0]
    enddate = rawdf['validdt'].iloc[-1]
    dailydf = pd.DataFrame(columns = ['date'], data = [dt.datetime(startdate.year, startdate.month, d) for d in range(1, enddate.day + 1)])

    #create cols for summary vars
    dailydf['mintmpf'] = -9999.
    dailydf['maxtmpf'] = -9999.
    dailydf['8amtmpf'] = -9999.
    dailydf['4pmtmpf'] = -9999.
    dailydf['maxdwpf'] = -9999.
    dailydf['maxsknt'] = -9999. #max sustained wind speed in knots
    dailydf['maxsdrct'] = -9999. #compass heading (wind blowing from) when max sustained wind was recorded
    dailydf['pctovc'] = -9999. #percent of day overcast
    dailydf['meanrh'] = -9999. #average relative humidity
    dailydf['minalt'] = -9999. #minimum altimer setting or air pressure
    dailydf['minvis'] = -9999. #minimum visibility
    dailydf['minfeel'] = -9999. #feels like temps
    dailydf['maxfeel'] = -9999.
    dailydf['pctpcp'] = -9999. #percent of day precipitating
    dailydf['pctmdt'] = -9999. #percent of day with moderate to heavy precip
    dailydf['pctsn'] = -9999. #percent of day snowing
    dailydf['amint'] = -9999 #precip intensity during AM rush hour (0: none, 1: light, 2: moderate, 3: heavy)
    dailydf['pmint'] = -9999 #precip intensity during PM rush hour

    for date in dailydf['date']:
        slc = rawdf.loc[rawdf.apply(lambda row: row['validdt'].day, axis=1) == date.day]
        idx = date.day - 1
        
        tmpf = pd.to_numeric(slc['tmpf'], errors='coerce')
        dailydf.at[idx, 'mintmpf'] = tmpf.min()
        dailydf.at[idx, 'maxtmpf'] = tmpf.max()
        #print(slc.apply(lambda row: row['validdt'].hour, axis=1) == 15)
        try:
            dailydf.at[idx, '8amtmpf'] = tmpf.iloc[15]
            dailydf.at[idx, '4pmtmpf'] = tmpf.iloc[31]
        except:
            continue
        
        dwpf = pd.to_numeric(slc['dwpf'], errors='coerce')
        dailydf.at[idx, 'maxdwpf'] = dwpf.max()
        
        sknt = pd.to_numeric(slc['sknt'], errors='coerce')
        dailydf.at[idx, 'maxsknt'] = sknt.max()
        drct = pd.to_numeric(slc['drct'], errors='coerce')
        dailydf.at[idx, 'maxsdrct'] = drct[sknt.idxmax()]

        dailydf.at[idx, 'pctovc'] = ((slc['skyc1'] == 'OVC') | (slc['skyc1'] == 'VV')).mean() * 100
        relh = pd.to_numeric(slc['relh'], errors='coerce')
        dailydf.at[idx, 'meanrh'] = relh.mean()

        alti = pd.to_numeric(slc['alti'], errors='coerce')
        dailydf.at[idx, 'minalt'] = alti.min()

        vsby = pd.to_numeric(slc['vsby'], errors='coerce')
        dailydf.at[idx, 'minvis'] = vsby.min()

        feel = pd.to_numeric(slc['feel'], errors='coerce')
        dailydf.at[idx, 'minfeel'] = feel.min()
        dailydf.at[idx, 'maxfeel'] = feel.max()

        try:
            dailydf.at[idx, 'pctpcp'] = slc.apply(lambda row: (row['wxcodes'] in pcptypes) | (row['wxcodes'][1:] in pcptypes), axis=1).mean() * 100
            dailydf.at[idx, 'pctmdt'] = slc.apply(lambda row: (row['wxcodes'] in pcptypes) | (row['wxcodes'][1:] in pcptypes) & (row['wxcodes'][0] == '+'), axis=1).mean() * 100
            dailydf.at[idx, 'pctsn'] = slc.apply(lambda row: 'SN' in row['wxcodes'], axis=1).mean() * 100
            dailydf.at[idx, 'amint'] = pcpint(slc['wxcodes'].iloc[15])
            dailydf.at[idx, 'pmint'] = pcpint(slc['wxcodes'].iloc[31])
        except:
            continue
    
    return dailydf

In [ ]:
#check computation of daily weather features
dailywx(df)

,date,mintmpf,maxtmpf,8amtmpf,4pmtmpf,maxdwpf,maxsknt,maxsdrct,pctovc,meanrh,minalt,minvis,minfeel,maxfeel,pctpcp,pctmdt,pctsn,amint,pmint
0,2022-04-01,26.6,41.0,37.4,39.2,26.6,11.0,70.0,6.250000,62.679167,29.94,6.21,18.20,35.75,0.000000,0.000000,0.000000,0,0
1,2022-04-02,21.2,42.8,23.0,39.2,28.4,9.0,340.0,0.000000,65.622000,30.03,6.21,17.84,40.03,0.000000,0.000000,0.000000,0,0
2,2022-04-03,28.4,44.6,41.0,44.6,41.0,18.0,280.0,0.000000,79.263651,29.62,2.05,22.12,38.39,25.396825,0.000000,0.000000,0,0
3,2022-04-04,35.6,46.4,39.2,41.0,44.6,24.0,250.0,1.190476,91.892500,28.94,0.81,26.58,38.98,70.238095,14.285714,1.190476,1,1
4,2022-04-05,32.0,42.8,39.2,41.0,33.8,22.0,280.0,0.000000,67.050526,29.26,6.21,25.53,34.78,3.508772,0.000000,1.754386,0,0
5,2022-04-06,33.8,44.6,33.8,33.8,42.8,17.0,120.0,29.487179,94.425897,29.00,0.93,22.43,44.60,66.666667,6.410256,21.794872,1,1
6,2022-04-07,35.6,44.6,41.0,41.0,41.0,18.0,230.0,21.052632,93.649868,28.61,1.74,26.58,41.00,50.000000,11.842105,0.000000,1,2
7,2022-04-08,32.0,41.0,32.0,35.6,35.6,16.0,270.0,0.000000,80.924262,28.88,0.31,21.16,33.84,31.147541,0.000000,26.229508,1,1
8,2022-04-09,35.6,46.4,41.0,42.8,35.6,27.0,290.0,0.000000,67.928482,29.15,6.21,27.02,38.06,0.000000,0.000000,0.000000,0,0
9,2022-04-10,37.4,44.6,39.2,39.2,33.8,23.0,280.0,0.000000,67.317545,29.56,6.21,27.46,36.58,0.000000,0.000000,0.000000,0,0


In [ ]:
#turn the raw half-hourly weather data into daily features and save
rawdir = './rawwx/'
for file in os.listdir(rawdir):
    filename = os.fsdecode(file)
    monthdf = pd.read_csv(rawdir + filename)
    print(filename)
    dailywx(monthdf).to_csv(path_or_buf = './dailywx/{}dailywx.csv'.format(filename[:-9]))

202201_EKAH_rawwx.csv
202202_EKAH_rawwx.csv
202203_EKAH_rawwx.csv
201101_EKAH_rawwx.csv
201102_EKAH_rawwx.csv
201103_EKAH_rawwx.csv
201104_EKAH_rawwx.csv
201105_EKAH_rawwx.csv
201106_EKAH_rawwx.csv
201107_EKAH_rawwx.csv
201108_EKAH_rawwx.csv
201109_EKAH_rawwx.csv
201110_EKAH_rawwx.csv
201111_EKAH_rawwx.csv
201201_EKAH_rawwx.csv
201202_EKAH_rawwx.csv
201203_EKAH_rawwx.csv
201204_EKAH_rawwx.csv
201205_EKAH_rawwx.csv
201206_EKAH_rawwx.csv
201207_EKAH_rawwx.csv
201208_EKAH_rawwx.csv
201209_EKAH_rawwx.csv
201210_EKAH_rawwx.csv
201211_EKAH_rawwx.csv
201301_EKAH_rawwx.csv
201302_EKAH_rawwx.csv
201303_EKAH_rawwx.csv
201304_EKAH_rawwx.csv
201305_EKAH_rawwx.csv
201306_EKAH_rawwx.csv
201307_EKAH_rawwx.csv
201308_EKAH_rawwx.csv
201309_EKAH_rawwx.csv
201310_EKAH_rawwx.csv
201311_EKAH_rawwx.csv
201401_EKAH_rawwx.csv
201402_EKAH_rawwx.csv
201403_EKAH_rawwx.csv
201404_EKAH_rawwx.csv
201405_EKAH_rawwx.csv
201406_EKAH_rawwx.csv
201407_EKAH_rawwx.csv
201408_EKAH_rawwx.csv
201409_EKAH_rawwx.csv
201410_EKA